In [1]:
!pip install -q FlagEmbedding
!pip install -q peft
!pip install -q faiss-gpu
!pip install openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 51.3 MB/s eta 0:00:00
   

In [2]:
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker

import os
import torch
import faiss
import re
import json
import numpy as np
import openai

from google.colab import userdata

import requests

In [3]:
# encode_queries()和encode()的区别：
# for s2p(short query to long passage) retrieval task, suggest to use encode_queries() which will automatically add the instruction to each query
# corpus in retrieval task can still use encode() or encode_corpus(), since they don't need instruction

model = FlagModel('BAAI/bge-base-en-v1.5', query_instruction_for_retrieval="Represent this sentence for searching relevant passages:", use_fp16=True)
# print(model.device)

reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True, device='cuda')
# print(reranker.device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [4]:
# query = "Tell me some information about the exams in the robotics class"
query = "What is the whether in Beijing?"

courses_numbers = ['544', '566', '585', '596', '599', '626', '677', '699']
course_names = ['Applied Natural Language Processing (NLP)', 'Deep Learning and its Applications (DL)', 'Database Systems (database)', 'Scientific Computing and Visualization',
         'Distributed Systems', 'Text as Data', 'Advanced Computer Vision (CV)', 'Robotic Perception (Robotics)']

full_course_info = dict(zip(['CSCI' + num for num in courses_numbers], ['CSCI' + num + ' ' + name for num, name in zip(courses_numbers, course_names)]))

name_to_num = dict(zip(course_names, courses_numbers))

full_course_info

{'CSCI544': 'CSCI544 Applied Natural Language Processing (NLP)',
 'CSCI566': 'CSCI566 Deep Learning and its Applications (DL)',
 'CSCI585': 'CSCI585 Database Systems (database)',
 'CSCI596': 'CSCI596 Scientific Computing and Visualization',
 'CSCI599': 'CSCI599 Distributed Systems',
 'CSCI626': 'CSCI626 Text as Data',
 'CSCI677': 'CSCI677 Advanced Computer Vision (CV)',
 'CSCI699': 'CSCI699 Robotic Perception (Robotics)'}

In [5]:
def build_vector_database(course_names):
    course_embeddings = model.encode(course_names)
    np_course_embeddings = np.array(course_embeddings).astype('float32')

    # with normalization -> bacome cosine similarity search
    faiss.normalize_L2(np_course_embeddings)

    # IndexFlatIP: dot-product search, IndexFlatL2: L2 search
    index_innerproduct = faiss.IndexFlatIP(len(course_embeddings[0]))
    res = faiss.StandardGpuResources()  # Create GPU resources
    gpu_index = faiss.index_cpu_to_gpu(res, 0, index_innerproduct) # Move index to GPU

    gpu_index.add(np_course_embeddings)

    return gpu_index

database = build_vector_database(course_names)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [6]:
def search_(query, database, embed_dim, topk=1):
    query_embed = model.encode_queries(query)
    query_embed = np.array(query_embed).astype('float32')

    _, idx = database.search(query_embed.reshape((1, embed_dim)), topk) # (768,) -> (1, 768)
    idx = idx.reshape(-1)

    ret = [course_names[i] for i in idx]

    return ret

def find_syllabus(query, courses_numbers, course_names, name_to_num, model, database):
    def find_course():
        for num in courses_numbers:
            if num in query:
                return num

        result = search_(query, database, embed_dim=database.d, topk=1)
        return result[0]

    result = find_course()

    if result in courses_numbers:
        return 'CSCI' + result + '.txt'
    else:
        return 'CSCI' + name_to_num[result] + '.txt'

In [7]:
print(find_syllabus(query, courses_numbers, course_names, name_to_num, model, database))

CSCI626.txt


In [8]:
def read_and_split_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    segments = content.split('\n\n##\n')
    # print(segments)

    return segments  # 变成了一个list

# 计算query和每个文档分段的相似度，并返回排序后的结果
def compute_similarity(query, segments):
    # encode_queries是专门用于encode query的
    query_embedding = model.encode_queries(query)  # [512, ]
    segment_embeddings = model.encode(segments)  # [num_seg, 512]

    # 官方提供的。只计算点积，不计算cos-simi
    similarities = [query_embedding @ segment_embedding.T for segment_embedding in segment_embeddings]

    # 将分数、query和文档分段组合并排序 -> 降序
    results = sorted(zip(similarities, segments), key=lambda x: x[0], reverse=True)
    # print(results)

    return results

# 定义生成查询和文档组合的函数
def generate_query_passage_pairs(query, passages):
    return [[query[0], passage] for passage in passages]

In [9]:
def RAG(**kwargs):
    query = kwargs.get('query', None)
    if query is None:
        raise ValueError('Your query cannot be empty!')

    k = 3

    final_syllabus = find_syllabus(query, courses_numbers, course_names, name_to_num, model, database)
    print(final_syllabus)

    file_path = '/content' + os.sep + final_syllabus
    knowledge_base = read_and_split_file(file_path)

    query = [query]
    all_similarity = compute_similarity(query, knowledge_base)

    results = []
    for score, segment in all_similarity:
        if score > 0.25:
            results.append(segment)

    query_passage_pairs = generate_query_passage_pairs(query, results)

    if query_passage_pairs:
        scores = reranker.compute_score(query_passage_pairs)
        sorted_query_passage_pairs = sorted(zip(scores, query_passage_pairs), key=lambda x: x[0], reverse=True)

        sorted_results = [(score, pair[1]) for score, pair in sorted_query_passage_pairs]

        final_results = []
        for score, passage in sorted_results:
            # if score > 0:
            final_results.append((score, passage))

        final_results = final_results[:k]
    else:
        final_results = [(0.0, 'There is no relevant information for the given query!')]

    all_segments = '\n\n'.join([result[1] for result in final_results])
    class_found = final_syllabus[:-len('.txt')]

    return f'Here are class information for {full_course_info[class_found]}:\n{all_segments}'

In [10]:
def financial_news_report(**kwargs):
    query = kwargs.get('query', None)
    if query is None:
        raise ValueError('Your query cannot be empty!')

    new_api_key = userdata.get('new_api')
    endpoint = 'https://newsapi.org/v2/everything'

    # Set query parameters
    params = {
        'q': 'financial OR stock market OR investment',  # Specify topics
        'language': 'en',     # Language preference
        'sortBy': 'publishedAt',  # Order by most recent
        'apiKey': new_api_key,
    }

    # Send request to News API
    response = requests.get(endpoint, params=params)

    # Check for successful response
    if response.status_code == 200:
        news_data = response.json()
        # Specify the maximum number of articles to retrieve
        max_articles = 5  # Set this to the number you want
        articles = news_data.get('articles', [])[:max_articles]  # Slice to limit the number
        fianl_res = ""
        # Parse and print each article's title and description
        for article in articles:
            print("Title:", article['title'])
            print("Description:", article['description'])
            fianl_res += "Title:" + article['title'] + "\n"
            fianl_res += "Description:" + article['description'] + "\n"

    else:
        print("Failed to retrieve news:", response.status_code)

    return fianl_res

In [11]:
def get_current_weather(**kwargs):
    # This would be replaced by a weather API
    # aa = f"What's the weather in {location}?"
    location = kwargs.get('location', None)
    if location is None:
        raise ValueError('Your location cannot be empty!')

    if location == "San Francisco, CA":
        return "The weather in San Francisco is 62 degrees and cloudy"
    elif location == "Philadelphia, PA":
        return "The weather in Philadelphia is 83 degrees and sunny"
    elif location == "Tokyo, Japan":
        return "The weather in Tokyo is 27 degrees and sunny"

    return f"The weather in {location} is unknown"

In [12]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "RAG",
            "description": "Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The original question that the user asks exactly, no need to rephrase.",
                    },
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Fetches the current weather for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city and country (e.g., 'San Francisco, US').",
                    },
                },
                "required": ["location"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "financial_news_report",
            "description": "Retrieve recent financial news when the user asks for it.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The original question that the user asks exactly, no need to rephrase.",
                    },
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        }
    },
]

In [13]:
# if you want to do multi-turn conversation -> messages.append({"role": "assistant", "content": "..."}); messages.append({"role": "user", "content": "..."})
messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. And only call one function at a time."})
messages.append({"role": "user", "content": query})

openai.api_key = userdata.get('OpenAI_API_Key')
response = openai.ChatCompletion.create(
    model="gpt-4",  # Specify the ChatGPT-4 model
    messages=messages,
    max_tokens=1024,
    temperature=0,
    tools=tools_list,
    tool_choice="auto"
)

print(response.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "tool_calls": [
    {
      "id": "call_jnY4YzDCJFR9t3jvMFtN7z33",
      "type": "function",
      "function": {
        "name": "get_current_weather",
        "arguments": "{\n  \"location\": \"Beijing\"\n}"
      }
    }
  ],
  "refusal": null
}


In [14]:
parsed_results = None

if response.choices[0].message.tool_calls is not None and response.choices[0].message.content is None:
    print('The model decide to call a function:\n')
    parsed_results = {}

    func_info = response.choices[0].message.tool_calls[0].function  # why [0]? because only call one function a time
    parsed_results['name'] = func_info.name
    parsed_results['arguments'] = json.loads(func_info.arguments)

    print(parsed_results['name'])

    for k, v in parsed_results['arguments'].items():
        print(f'{k}: {v}')
else:
    print("No function call found in the response:\n")
    print(response.choices[0].message.content)

The model decide to call a function:

get_current_weather
location: Beijing


In [15]:
if parsed_results is not None:
    available_functions = {
        "RAG": RAG,
        "get_current_weather": get_current_weather,
        "financial_news_report": financial_news_report,
    }

    if parsed_results["name"] not in available_functions:
        available_function_names = "\n".join(available_functions.keys())
        raise NotImplementedError(
            f"Function {parsed_results['name']} is not implemented. "
            f"Our available functions are:\n\n{available_function_names}"
        )

    function_to_call = available_functions[parsed_results['name']]

    # result, final_course = function_to_call(query)
    result = function_to_call(**parsed_results["arguments"])
    print(result)
    print()

    answer_prompt = "You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question."

    new_messages = [
        {
            "role": "system",
            "content": answer_prompt,
        },
        {
            "role": "user",
            "content": f"""
## Question:
{query}

## Information:
{result}
"""
        },
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4",  # Specify the ChatGPT-4 model
        messages=new_messages,
        max_tokens=1024,
    )

The weather in Beijing is unknown



In [16]:
print(answer_prompt)
print(new_messages[1]['content'])

You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question.

## Question:
What is the whether in Beijing?

## Information:
The weather in Beijing is unknown



In [17]:
print(response.choices[0].message.content)

The current weather in Beijing is unknown.
